# Debugging gaussian state preparation

In [1]:
import numpy as np

import copy
from matplotlib import pyplot as plt
from qat.qpus import get_default_qpu

from qat.fermion.hamiltonians import make_hubbard_model, ElectronicStructureHamiltonian
from qat.fermion.matchgates import get_nn_rotation_angles, prepare_R_matrix
from qat.fermion.circuits import make_ldca_circ
from qat.dqs.fermionic_util import init_creation_ops, dag

from qat.core.simutil import wavefunction
from gaussian_util import herm_conj, generate_unitary_matrix, make_quadratic_hamiltonian, make_rotation_thouless, compute_diff_to_thouless
from qat.qpus import LinAlg

from scipy.stats import unitary_group

## 1. Checking the equivalence between the single-particle modes rotation and the corresponding conjugate action in Fock space
Here, we want to check that $ \tilde{H} = \mathcal{U} H \mathcal{U}^{\dagger}$ with $H = \sum_{pq} h_{pq} a^{\dagger}_p a_q$, $\tilde{h} = r h r^{\dagger}$, $H = \sum_{pq} \tilde{h}_{pq} a^{\dagger}_p a_q$, $\mathcal{U} = \exp \left( \sum_{pq} \log(r)_{pq} (a^{\dagger}_p a_q - \mathrm{h.c.} )\right)$.
For that, we generate at random a unitary matrix, using either the custom function or scipy's. Then, we feed it to a function that will check that the relation above holds by picking at random a hermitian matrix and constructing the corresponding Hamiltonian and the corresponding rotated Hamiltonian.

In [2]:
r = generate_unitary_matrix(2)
#r = unitary_group.rvs(2) # alternative

diff = compute_diff_to_thouless(r, antisymmetrize=True)
print('Norm of operator difference with AS=', diff)

diff = compute_diff_to_thouless(r, antisymmetrize=False)
print('Norm of operator difference without AS=', diff)

Norm of operator difference with AS= 0.5324550915906859
Norm of operator difference without AS= 1.4663247218518516e-15


## 2. Connecting ground states of quadratic Hamiltonians
Here, we consider a quadratic Hamiltonian, diagonalize it and look at its ground state. We observe it corresponds to the filling of negative-energy orbitals.

In [3]:
# Generate quadratic Hamiltonian
M = 4
np.random.seed(seed=0)
mat = np.random.random((M, M))
hpq = mat + herm_conj(mat)

H = ElectronicStructureHamiltonian(hpq=hpq, hpqrs=np.zeros((M, M, M, M)), constant_coeff=0)
H_mat = H.get_matrix()
eigvals, eigvecs = np.linalg.eigh(H_mat)
E0 = eigvals[0]
psi0 = eigvecs[:, 0]
print('E0=', E0)
diag_energies, r = np.linalg.eigh(hpq)
print(diag_energies)

H_tilde = ElectronicStructureHamiltonian(hpq=herm_conj(r).dot(hpq.dot(r)), hpqrs=np.zeros((M, M, M, M)), constant_coeff=0)
H_tilde_mat = H_tilde.get_matrix()
eigvals_tilde, eigvecs_tilde = np.linalg.eigh(H_tilde_mat)

psi0_tilde = eigvecs_tilde[:, 0]

print('psi0 tilde=', psi0_tilde)

E0= -1.5567715973003715
[-1.21115868 -0.34561291  1.08987198  4.61402353]
psi0 tilde= [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


Let $r$ be the Hamiltonian's diagonalization matrix. We construct $\mathcal{U}(r)$. Let $|\psi_0\rangle$ be the ground state of the original Hamiltonian $H$, $|\tilde{\psi}_0\rangle$ that of the diagonalized Hamiltonian $\tilde{H}$. We expect that $|\psi_0\rangle = \mathcal{U}(r)| \tilde{\psi}_0\rangle$.

In [4]:
U_cal = make_rotation_thouless(r)
transformed_psi0 = U_cal.dot(psi0_tilde)

# checking that normalization is conserved (sanity check, as U_cal is checked by its constructor for unitarity)
print(np.linalg.norm(transformed_psi0))
print(herm_conj(psi0).dot(transformed_psi0))
print(np.linalg.norm(psi0 + transformed_psi0))

0.9999999999999996
(-1+0j)
2.136079133711712e-15


In [5]:
""""
FROM THERE: WIP
""""
assert 0==1

SyntaxError: EOL while scanning string literal (3240389896.py, line 3)

In [ ]:
a_dag = init_creation_ops(M)
psi_ref = np.zeros((2**M,))
psi_ref[0] = 1
psi0_tilde_bis = a_dag[0].dot(a_dag[1].dot(psi_ref))
print(psi0_tilde_bis)

In [ ]:
givens_rotation_angles, cf = get_nn_rotation_angles(r)
print(cf)

In [ ]:
R = prepare_R_matrix(2, givens_rotation_angles)
print(R)
U_cal_bis = make_rotation_thouless(R)
print(np.linalg.norm(U_cal - U_cal_bis))

In [ ]:
circ = make_ldca_circ(nb_fermionic_modes=4, ncycles=1)

nb_ldca_params = len(circ.var_dic)

nb_angles = len(givens_rotation_angles)
nb_other_params = nb_ldca_params - nb_angles

# We set the initial parameters to the Givens rotations, the rest of the parameters beeing set to 0
theta = list(givens_rotation_angles) + [0]*nb_other_params 

# We bind these parameters to the paramterized circuit defined previously
prep_circ = circ.bind_variables({ r"\theta_{%i}"%i : 0.5*theta[i] for i in range(len(theta))}) # beware of the 0.5 factor!

In [ ]:
circ_wf = wavefunction(prep_circ, LinAlg())

In [ ]:
print(herm_conj(psi0).dot(circ_wf))